# Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt

# Load Data

In [ ]:
import pandas as pd

# Load the dataset
file_path = r'C:\Users\File.csv'
#---------------------------------------------
data = pd.read_csv(file_path)

# Drop rows where 'ISP' is missing to retain only rows with servers
data = data.dropna(subset=['Org'])

# Make 'Adjusted Time' the first column by reordering
columns = ['AdjustedTime'] + [col for col in data.columns if col != 'AdjustedTime']
data = data[columns]

# Data Mapping

In [ ]:
# Map each unique 'Source' IP to a server ID
unique_servers = data['Source'].unique()
server_mapping = {ip: idx for idx, ip in enumerate(unique_servers, start=1)}
data['ServerID'] = data['Source'].map(server_mapping)
data['ServerID'] -= 1

# Select features and the new target (ServerID)
features = data[['AdjustedTime', 'Protocol', 'Connection', 'User', 'Length', 'ARTT', 'Longitude', 'Latitude', 'Org']]
label = data['ServerID']

# Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Convert datetime to numeric
data['AdjustedTime'] = pd.to_datetime(data['AdjustedTime'])
data['AdjustedTimestamp'] = data['AdjustedTime'].astype(np.int64) // 10**9

# Label encode categorical features
for col in ['Protocol', 'Connection', 'User', 'Org']:
    data[col] = LabelEncoder().fit_transform(data[col])

# Define features
features = ['AdjustedTimestamp', 'Protocol', 'Connection', 'User', 'Length', 'ARTT', 'Longitude', 'Latitude', 'Org']
X = data[features]
y = data['ServerID']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# DNN Model Defintion

In [ ]:
# Define the DNN model with at least 3 layers
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001), input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(16, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(len(unique_servers), activation='softmax')
])


# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Add EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   # Monitor validation loss
    patience=10,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True # Restore the best model weights after stopping
)

# Training

In [ ]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.176, batch_size=32, callbacks=[early_stopping], verbose=1)

# Evaluate

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print("Model accuracy on test set:", accuracy)